# Morse Complex
Tast of today's lab exercises is to implement an algorithm that computes a Morse Complex of a given simplicial complex K.

To make things simple the simplicial complex K is given as a list of simplices and each simplex is just a tuple of its vertices. 

Example: bellow is the definition of a complex representing a full triangle spanned on the vertices $0, 1$ and $2$.

In [1]:
K = [(0,), (1,), (2,), (0,1), (0,2), (1, 2), (0, 1, 2)] 

Algorithm is implemented in 3 steps:
1. Compute a random discrete gradient vector field on K.
2. Cancel unnecessary critical simplices.
3. Compute the Morse boundary of each critical simplex.

### Compute a random discrete gradient vector field on K
Use the procedure described in the article by [Bruno Benedetti and Frank
Lutz](http://arxiv.org/abs/1303.6422). The procedure is implemented bellow.

In [2]:
from random import choice
from collections import defaultdict


def contained(a, b):
    """Returns True is a is a subsimplex of b, False otherwise."""
    return all((v in b for v in a))

def free_face(K):
    """Finds one of the free faces in the complex K.
    Returns the pair (a, b) where a is a free face of b.
    If K has no free faces, tuple (None, None) is returned.  
    """
    for s1 in K:
        cofaces = [s for s in K if len(s) == len(s1) + 1 and contained(s1, s)]
        if len(cofaces) == 1: 
            return s1, cofaces[0]
    return None, None

def top_simplex(K):
    """Returns one of the top dimensional simplices in K.  
    """
    max_length = max([len(s) for s in K])
    return choice([s for s in K if len(s) == max_length])
                
def generate_field(K):
    """Computes a discrete gradient vector field on the complex K.
    Returns a pair (V, C), where V representing pairs in the
    computed gradient vector field and C is a list of critical 
    cimplices.  
    """
    K1, C, V = set(K), [], dict()
    while K1:
        s1, s2 = free_face(K1)
        if s1:
            V[s1] = s2
            K1.remove(s1)
            K1.remove(s2)            
        else:
            s = top_simplex(K1)
            C.append(s)
            K1.remove(s)
    return V, C

Let us try the code on the complex K (full triangle).

In [3]:
V, C = generate_field(K)
print(V, C)

({(0, 1): (0, 1, 2), (0,): (0, 2), (1,): (1, 2)}, [(2,)])


### Cancel unnecessary critical simplices
First we have to find all paths starting in the boundary of some critical simplex and ending in the other critical cell. Then we can cancel out critical cells that are connected with exactly one gradient path. Repeat the procedure until all such pairs are canceled.

In [76]:
from itertools import combinations
from collections import defaultdict

def simplex_closure(a):
    """Returns the generator iteration over all subsimplices 
    (of all dimensions) of the simplex a.
    The simplex a is not included in the returned generator.
    """
    faces = []
    for l in range(1, len(a)):
        faces += combinations(a, l)
    return faces

def closure(K):
    """Add all simplices to K in order to make it a simplicial complex."""
    K1 = set(K)
    for s in K:
        K1.update(simplex_closure(s))
    return list(K1)

def boundary(a):
    """Return the co-dimension one faces of the simplex a.
    """
    return combinations(a, len(a)-1)

def expand_path(K, V, C, path):
    """Expands a path with the given prefix which is stored in the
    list path. The last simplex in the list path in the end of the 
    prefix.
    Returns the list of all paths with the given prefix which end in
    critical cells.
    """
    end = path[-1]
    if end in C:      # prefix already ends at the critical simplex
        return [path]
    if end not in V:  # path ends here
        return []
    paths = []
    n = V[end]
    for child in [child for child in boundary(n) if child != end]:
        paths += expand_path(K, V, C, path + [n, child])
    return paths

def find_paths(K, V, C):
    """Find all paths starting in the boundary of critical simplices 
    (of dimension 1 or greater) and ending in other critical cell.
    Returns dictionary whose keys are critical simplices and value
    for the given key is a list of such paths starting in the boundary 
    the key.
    """
    paths = defaultdict(list)
    candidates = [s for s in C if len(s) > 1]
    for candidate in candidates:
        for start in boundary(candidate):
            paths[candidate] += expand_path(K, V, C, [start])
    return paths

def cancel(V, C, start, path):
    reversed_path = list(reversed(path)) + [start]
    for i in range(0, len(reversed_path), 2):
        V[reversed_path[i]] = reversed_path[i+1]
    C.remove(start)
    C.remove(path[-1])
        

def cancel_all(K, V, C):
    candidates = [start for start, path in find_paths(K, V, C).items() if len(path) == 1]
    while candidates:
        candidate = choose(candidates)
        cancel(V, C, candidate, paths[candidate])
        candidates = [start for start, path in find_paths(K, V, C).items() if len(path) == 1]

Let us try the code on a more complex example.

In [75]:
K = closure([(0, 1, 4), (0, 3, 4), (0, 2, 3), (2, 3, 5), (1, 2, 5), (1, 4, 5)])
#C = [(0, 2, 3), (0, 1), (1, 4), (0, 2), (3, 5), (4,), (2,), (0,)]
V, C = generate_field(K)
cancel_all(K, V, C)